# Opnepose로 Keypoint 뽑기(body_25)

In [1]:
import os, sys

In [2]:
#!pip install opencv-python

In [3]:
#!pip uninstall opencv-python
#%cd /content/drive/MyDrive/cv2_gpu
#!cp cv2.cpython-37m-x86_64-linux-gnu.so /usr/local/lib/python3.6/dist-packages

In [4]:
import cv2
import os, sys
#from google.colab.patches import cv2_imshow

#PATH = '/content/drive/MyDrive/openpose-master/models/pose/mpi'
print(cv2.__version__)
print(cv2.cuda.getCudaEnabledDeviceCount())

4.6.0
0


#bodypart 

In [5]:
BODY_PARTS_BODY_25 = {0: "Nose", 1: "Neck", 2: "RShoulder", 3: "RElbow", 4: "RWrist",
                      5: "LShoulder", 6: "LElbow", 7: "LWrist", 8: "MidHip", 9: "RHip",
                      10: "RKnee", 11: "RAnkle", 12: "LHip", 13: "LKnee", 14: "LAnkle",
                      15: "REye", 16: "LEye", 17: "REar", 18: "LEar", 19: "LBigToe",
                      20: "LSmallToe", 21: "LHeel", 22: "RBigToe", 23: "RSmallToe", 24: "RHeel", 25: "Background"}

POSE_PAIRS_BODY_25 = [[0, 1], [0, 15], [0, 16], [1, 2], [1, 5], [1, 8], [8, 9], [8, 12], [9, 10], [12, 13], [2, 3],
                      [3, 4], [5, 6], [6, 7], [10, 11], [13, 14], [15, 17], [16, 18], [14, 21], [19, 21], [20, 21],
                      [11, 24], [22, 24], [23, 24]]

In [6]:
def getDistance(a, b):
  ax, ay = a
  bx, by = b

  dist = float((ax-bx)**2 + (ay-by)**2)**(0.5)
  return dist 

getDistance((0, 4), (3, 0))

5.0

In [7]:
# 각 파일 path
protoFile = "body25/pose_deploy.prototxt"
weightsFile = "body25/pose_iter_584000.caffemodel"

#imageFolder = "/content/drive/MyDrive/people_pose"
 
# 위의 path에 있는 network 불러오기
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
#net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
#net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

In [8]:
def pointCoord(image):
    Image = image
    imageHeight, imageWidth, _ = Image.shape
  
  # network에 넣기위해 전처리
    inpBlob = cv2.dnn.blobFromImage(image, 1.0 / 255, (imageWidth, imageHeight), (0, 0, 0), swapRB=False, crop=False)
  
  # network에 넣어주기
    net.setInput(inpBlob)

  # 결과 받아오기
    output = net.forward()

  # output.shape[0] = 이미지 ID, [1] = 출력 맵의 높이, [2] = 너비
    H = output.shape[2]
    W = output.shape[3]
  # print("이미지 ID : ", len(output[0]), ", H : ", output.shape[2], ", W : ",output.shape[3]) # 이미지 ID

  # 키포인트 검출시 좌표 추가
    points = []
    for i in range(len(BODY_PARTS_BODY_25)):
      # 해당 신체부위 신뢰도 얻음.
        probMap = output[0, i, :, :]
  
      # global 최대값 찾기
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
        

      # 원래 이미지에 맞게 점 위치 변경
        x = (imageWidth * point[0]) / W
        y = (imageHeight * point[1]) / H

      # 키포인트 검출한 결과가 0.1보다 크면(검출한곳이 위 BODY_PARTS랑 맞는 부위면) points에 추가, 검출했는데 부위가 없으면 None으로    
        #print(BODY_PARTS_BODY_25)
        if prob > 0.1 :    
           # print(f"[pointed] {BODY_PARTS_BODY_25[i]} ({i}) => prob: {prob:.5f} / x: {x} / y: {y}\n")
            points.append((int(x), int(y)))
        else :
            points.append(None)

    return points

In [9]:
class WalkFrame:
    points = ()
    frame = None
    fps = None
    Rvelocity, Lvelocity = 0.0, 0.0
    Rflag, Lflag = False, False
    def __init__(self, frame, fps):
        self.frame = frame
        print("-----run-----")
        self.points = pointCoord(self.frame)
        self.fps = fps
    
    def getPoints(self):
        return self.points

    def getVelocity(self, otherf):
        if self.points[10] is None or otherf.points[10] is None:
            self.Rvelocity = None
        else:
            self.Rvelocity = getDistance(self.points[10] , otherf.points[10]) / self.fps

        if self.points[13] is None or otherf.points[13] is None:
            Lvelocity = None
        else:
            self.Lvelocity = getDistance(self.points[13] , otherf.points[13]) / self.fps
    #왼발과 오른발의 속도를 얻음
        if self.Rvelocity == 0.0:
            Rflag = True 
        if self.Lvelocity == 0.0:
            Lflag = True

    def drawIt(self):
        color = (0, 255, 255) #기본값은 노랑색
        red = (0, 0, 255)

        for i in range(0, 26):
            coord = self.points[i]
      
            if self.points[i] is not None:
                if (i==10 and self.Rflag) or (i==13 and self.Lflag):
                    cv2.circle(self.frame, coord, 3, (0,0,255), thickness=-1, lineType=cv2.FILLED)

                else:
                    cv2.circle(self.frame, coord, 3, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
                #왼발이나 오른발이 기준보다 속도가 느리다면, 점의 색상을 빨간색으로 바꿈.

        

            else:
                pass

        #for pair in POSE_PAIRS_BODY_25:
         #   partA = pair[1]             
          #  partB = pair[0]                   # 각 파트의 이름
          #  partA = BODY_PARTS_BODY_25[partA]   
          #  partB = BODY_PARTS_BODY_25[partB]   # 각 파트에 해당하는 번호
      
      #print(partA," 와 ", partB, " 연결\n")
            #if self.points[partA] and self.points[partB]:
             #   cv2.line(self.frame, self.points[partA], self.points[partB], (0, 255, 0), 2)


In [10]:
#각 프레임마다 좌표를 찍어 밖으로 내보냄

def pointing(vdo):
    video = vdo
    video_frame=video.get(cv2.CAP_PROP_FPS)
    i=0
    result_points = []

    CoordImage = [] # 좌표찍은 프레임들
    while video.isOpened():
        run, frame = video.read()
        if not run:
            print("종료")
            break
        CoordImage.append(WalkFrame(frame, video_frame))
        result_points.append(CoordImage[i].getPoints())
        #print(result_points)
        i=i+1
        print(i)
    
    

    for i in range(1, len(CoordImage)):
        CoordImage[i].getVelocity(CoordImage[i-1])
        

    return CoordImage, result_points

In [11]:
#for test

imageFolder = "/result"
videoPath = "walk_side.mp4"
Video = cv2.VideoCapture(videoPath)
video_width=Video.get(cv2.CAP_PROP_FRAME_WIDTH)
video_height=Video.get(cv2.CAP_PROP_FRAME_HEIGHT)
video_frame=Video.get(cv2.CAP_PROP_FPS)

frames, points_list = pointing(Video)

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
output = cv2.VideoWriter("testwalk.mp4", fourcc, int(video_frame), (int(video_width), int(video_height)))
for frame in frames:
  frame.drawIt()
  print(frame.Rflag, frame.Lflag)
  output.write(frame.frame)
output.release() #마무리하고 내보냄





-----run-----
1
-----run-----
2
-----run-----
3
-----run-----
4
-----run-----
5
-----run-----
6
-----run-----
7
-----run-----
8
-----run-----
9
-----run-----
10
-----run-----
11
-----run-----
12
-----run-----
13
-----run-----
14
-----run-----
15
-----run-----
16
-----run-----
17
-----run-----
18
-----run-----
19
-----run-----
20
-----run-----
21
-----run-----
22
-----run-----
23
-----run-----
24
-----run-----
25
-----run-----
26
-----run-----
27
-----run-----
28
-----run-----
29
-----run-----
30
-----run-----
31
-----run-----
32
-----run-----
33
-----run-----
34
-----run-----
35
-----run-----
36
-----run-----
37
-----run-----
38
-----run-----
39
-----run-----
40
-----run-----
41
-----run-----
42
-----run-----
43
-----run-----
44
-----run-----
45
-----run-----
46
-----run-----
47
-----run-----
48
-----run-----
49
-----run-----
50
-----run-----
51
-----run-----
52
-----run-----
53
-----run-----
54
-----run-----
55
-----run-----
56
-----run-----
57
-----run-----
58
-----run-----
59
-----r

In [12]:
import pandas as pd
import matplotlib as plt
import numpy as np

In [13]:
point_dataframe = pd.DataFrame(columns = ["Nose_x", "Nose_y", "Neck_x", "Neck_y",
                             "RShoulder_x","RShoulder_y", "RElbow_x", "RElbow_y",
                             "RWrist_x", "RWrist_y","LShoulder_x", "LShoulder_y",
                             "LElbow_x", "LElbow_y", "LWrist_x", "LWrist_y",
                             "MidHip_x", "MidHip_y", "RHip_x", "RHip_y",
                             "RKnee_x", "RKnee_y", "RAnkle_x", "RAnkle_y", 
                             "LHip_x", "LHip_y", "LKnee_x", "LKnee_y",
                             "LAnkle_x", "LAnkle_y","REye_x", "REye_y",
                             "LEye_x", "LEye_y", "REar_x", "REar_y",
                             "LEar_x", "LEar_y", "LBigToe_x", "LBigToe_y",
                             "LSmallToe_x", "LSmallToe_y", "LHeel_x", "LHeel_y",
                             "RBigToe_x", "RBigToe_y", "RSmallToe_x", "RSmallToe_y",
                             "RHeel_x", "RHeel_y", "Background_x", "Background_y"])
points_list_for_dataframe = []
for points in points_list:
    points_by_frame = []
    for i in points:
        if type(i) == type(None):
            points_by_frame.append(None)
            points_by_frame.append(None)
        else:
            points_by_frame.append(i[0])
            points_by_frame.append(i[1])
    #print(points_by_frame)
    points_list_for_dataframe.append(points_by_frame)


    
    #point_dataframe = point_dataframe.append(pd.Series(points_by_frame), ignore_index = True)
point_dataframe = pd.DataFrame(data = points_list_for_dataframe, columns = ["Nose_x", "Nose_y", "Neck_x", "Neck_y",
                             "RShoulder_x","RShoulder_y", "RElbow_x", "RElbow_y",
                             "RWrist_x", "RWrist_y","LShoulder_x", "LShoulder_y",
                             "LElbow_x", "LElbow_y", "LWrist_x", "LWrist_y",
                             "MidHip_x", "MidHip_y", "RHip_x", "RHip_y",
                             "RKnee_x", "RKnee_y", "RAnkle_x", "RAnkle_y", 
                             "LHip_x", "LHip_y", "LKnee_x", "LKnee_y",
                             "LAnkle_x", "LAnkle_y","REye_x", "REye_y",
                             "LEye_x", "LEye_y", "REar_x", "REar_y",
                             "LEar_x", "LEar_y", "LBigToe_x", "LBigToe_y",
                             "LSmallToe_x", "LSmallToe_y", "LHeel_x", "LHeel_y",
                             "RBigToe_x", "RBigToe_y", "RSmallToe_x", "RSmallToe_y",
                             "RHeel_x", "RHeel_y", "Background_x", "Background_y"])


In [14]:
point_dataframe

,Nose_x,Nose_y,Neck_x,Neck_y,RShoulder_x,RShoulder_y,RElbow_x,RElbow_y,RWrist_x,RWrist_y,...,LHeel_x,LHeel_y,RBigToe_x,RBigToe_y,RSmallToe_x,RSmallToe_y,RHeel_x,RHeel_y,Background_x,Background_y
0,160,40,120,80,112,80,72,152,72.0,232.0,...,96.0,424.0,120.0,432.0,112.0,432.0,88.0,424.0,168,8
1,160,40,120,88,112,88,72,160,72.0,232.0,...,96.0,424.0,120.0,440.0,112.0,440.0,80.0,424.0,104,120
2,160,40,128,88,112,88,80,168,80.0,240.0,...,96.0,416.0,128.0,432.0,88.0,432.0,72.0,424.0,96,120
3,168,40,128,88,112,88,88,168,88.0,240.0,...,96.0,424.0,NaN,NaN,NaN,NaN,72.0,424.0,96,120
4,168,40,128,88,120,88,104,168,104.0,240.0,...,120.0,416.0,NaN,NaN,NaN,NaN,NaN,NaN,104,120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,272,48,240,96,232,96,216,176,232.0,248.0,...,216.0,432.0,NaN,NaN,NaN,NaN,216.0,432.0,128,8
135,272,48,240,96,232,96,216,176,232.0,248.0,...,216.0,432.0,NaN,NaN,NaN,NaN,216.0,432.0,128,8
136,272,48,240,96,232,96,216,176,232.0,248.0,...,NaN,NaN,NaN,NaN,NaN,NaN,216.0,424.0,128,8
137,272,48,240,96,232,96,216,168,240.0,240.0,...,NaN,NaN,NaN,NaN,NaN,NaN,216.0,432.0,264,296


In [15]:
point_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 52 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Nose_x        139 non-null    int64  
 1   Nose_y        139 non-null    int64  
 2   Neck_x        139 non-null    int64  
 3   Neck_y        139 non-null    int64  
 4   RShoulder_x   139 non-null    int64  
 5   RShoulder_y   139 non-null    int64  
 6   RElbow_x      139 non-null    int64  
 7   RElbow_y      139 non-null    int64  
 8   RWrist_x      138 non-null    float64
 9   RWrist_y      138 non-null    float64
 10  LShoulder_x   139 non-null    int64  
 11  LShoulder_y   139 non-null    int64  
 12  LElbow_x      85 non-null     float64
 13  LElbow_y      85 non-null     float64
 14  LWrist_x      85 non-null     float64
 15  LWrist_y      85 non-null     float64
 16  MidHip_x      139 non-null    int64  
 17  MidHip_y      139 non-null    int64  
 18  RHip_x        139 non-null    